In [62]:
import requests

api_key = "up_X0eMHEuvQ2adWZHewcmbzZOliKVN2"
filename = "/Users/livdea/Downloads/sample_presc1.png"

url = "https://api.upstage.ai/v1/document-ai/document-parse"
# url = "https://api.upstage.ai/v1/document-ai/ocr"

headers = {"Authorization": f"Bearer {api_key}"}
files = {"document": open(filename, "rb")}
response = requests.post(url, headers=headers, files=files)
response.json()

{'api': '2.0',
 'content': {'html': '<h1 id=\'0\' style=\'font-size:20px\'>Dr. Onkar Bhave<br>M.B.B.S., M.D., M.S.  Reg. No: 270988|<br>Mob. No: 8983390126</h1>\n<br><figure><img id=\'1\' style=\'font-size:16px\' alt="\nLOREM IPSUM\n\'" data-coord="top-left:(239,1); bottom-right:(293,52)" /></figure>\n<br><h1 id=\'2\' style=\'font-size:22px\'>Care Clinic</h1>\n<br><p id=\'3\' data-category=\'paragraph\' style=\'font-size:18px\'>Near Axis Bank, Kothrud, Pune-<br>411038.<br>Ph: 094233 80390, Timing: 09:00<br>AM = 02:00 PM  Closed: Thursday</p>\n<h1 id=\'4\' style=\'font-size:18px\'>ID: 266 - DEMO PATIENT (M)<br>Address: PUNE<br>Temp (deg): 36, BP: 120/80 mmHg</h1>\n<br><h1 id=\'5\' style=\'font-size:18px\'>Date: 27-Apr-2020, 04:37 PM</h1>\n<figure><img id=\'6\' style=\'font-size:14px\' alt="R\nMedicine Name Dosage Duration\n1) TAB. DEMO MEDICINE 1 1 Morning, 1 Night 10 Days\n(Before Food) (Tot:20 Tab)\n2) CAP. DEMO MEDICINE 2 1 Morning, 1 Night 10 Days\n(Before Food) (Tot:20 Cap)\n3) TAB

In [63]:
import together
together_client = together.Together(api_key='63f7c168bb7791313bf41a29c742dbf3ca9d5930ff1bab9ba4f3b9a7aeb25c10')


In [75]:
prompt = f"""
You will be provided a context parsed from a physician's prescription. 
CONTEXT: {response.json()}
TASK: Extract the information about the prescribed medicine names in the following JSON format:
Example: 
'medication_names': ['Medication 1', 'Medication 2'],
'dosage': ['1 Morning, 1 Night', '1 Afternoon'],
'dose_time':[['10:00', '21:00'], ['14:00']], # Time of the day when the dose is taken as per dosage; keep 10:00 for morning, 14:00 for afternoon, 21:00 for night
'total_dose_per_day': [2, 1], # Sum up the dosage
'before/after meal':['after meal', 'before meal'],
'duration': ['10 days', '2 months']
Provide the output in the exact format as shown in the example. Do not include any additional text, explanations, or code.
"""

output = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}]
    )

In [76]:
print(output.choices[0].message.content)

{
'medication_names': ['TAB. DEMO MEDICINE 1', 'CAP. DEMO MEDICINE 2', 'TAB. DEMO MEDICINE 3', 'TAB. DEMO MEDICINE 4'],
'dosage': ['1 Morning, 1 Night', '1 Morning, 1 Night', '1 Morning, 1 Afternoon, 1 Evening, 1 Night', '1/2 Morning, 1/2 Night'],
'dose_time': [['10:00', '21:00'], ['10:00', '21:00'], ['10:00', '14:00', '18:00', '21:00'], ['10:00', '21:00']],
'total_dose_per_day': [2, 2, 4, 1.5],
'before/after meal': ['before meal', 'before meal', 'after meal', 'after meal'],
'duration': ['10 days', '10 days', '10 days', '10 days']
}


In [77]:
import ast

prescription_info = ast.literal_eval(output.choices[0].message.content)
prescription_info

{'medication_names': ['TAB. DEMO MEDICINE 1',
  'CAP. DEMO MEDICINE 2',
  'TAB. DEMO MEDICINE 3',
  'TAB. DEMO MEDICINE 4'],
 'dosage': ['1 Morning, 1 Night',
  '1 Morning, 1 Night',
  '1 Morning, 1 Afternoon, 1 Evening, 1 Night',
  '1/2 Morning, 1/2 Night'],
 'dose_time': [['10:00', '21:00'],
  ['10:00', '21:00'],
  ['10:00', '14:00', '18:00', '21:00'],
  ['10:00', '21:00']],
 'total_dose_per_day': [2, 2, 4, 1.5],
 'before/after meal': ['before meal',
  'before meal',
  'after meal',
  'after meal'],
 'duration': ['10 days', '10 days', '10 days', '10 days']}

In [67]:
prescription_info.keys()

dict_keys(['medication_names', 'dosage', 'total_dose_per_day', 'before/after meal', 'duration'])

In [61]:
prescription_info['medication_names']

['TAB. DEMO MEDICINE 1',
 'CAP. DEMO MEDICINE 2',
 'TAB. DEMO MEDICINE 3',
 'TAB. DEMO MEDICINE 4']

In [8]:
import base64

# Convert image to base64
with open(filename, "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

In [24]:
from openai import OpenAI # openai==1.2.0
 
client = OpenAI(
    api_key="up_X0eMHEuvQ2adWZHewcmbzZOliKVN2",
    base_url="https://api.upstage.ai/v1/solar"
)
 
response = client.chat.completions.create(
    model="solar-docvision",
    messages=[
        {
          "role": "user",
          "content": [
              {
                  "type": "image_url",
                  "image_url": {
                      "url": f"data:image/png;base64,{encoded_string}"
                  },
              },
              {
                  "type": "text",
                  "text": """Extract the information about the prescribed medicine names in the following format:
                  Example: 
                  {'medication_names': ['Medication 1', 'Medication 2'],
                  'dosage': ['1 Morning, 1 Night', '1 Morning'],
                  'total_dose_per_day': [2, 1],
                  'before/after meal':['after meal', 'before meal'],
                  'duration': ['10 days', '2 months']}
                  """
              },
          ]
        }
    ],
)
 
print(response.choices[0].message.content)
 
# Use with stream=True
# for chunk in response:
#     if chunk.choices[0].delta.content is not None:
#         print(chunk.choices[0].delta.content, end="")

 {'medication_names': ['Tab. Demo Medicine 1', 'Cap. Demo Medicine 2', 'Tab. Demo Medicine 3', 'Tab. Demo Medicine 4'],
 'dosage': ['1 Morning, 1 Night', '1 Morning', '1 Morning, 1 Night, 1 Eve, 1 Night', '1 Morning, 1/2 Night'],
 'total_dose_per_day': [2, 1, 3, 2],
 'before/after meal':['after meal', 'before meal', 'after meal', 'after meal'],
 'duration': ['10 days', '10 days', '10 days', '10 days']}




In [ ]:
{m1:{dosage: '1 Morning', total_dose_per_day: 1, before/after meal: 'after meal', duration: '10 days'},
m2:{dosage: '1 Morning', total_dose_per_day: 1, before/after meal: 'after meal', duration: '10 days'}}